# Network Plotting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

from sknetwork.data import from_edge_list
import networkx as nx

In [2]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure, from_networkx

output_notebook()

Loading BokehJS ...

In [3]:
DIR = Path.cwd() / "data" / "consensusResults2"

In [4]:
consensusResults = np.load(f"{DIR}/consensusResults.npy", allow_pickle=True)

In [5]:
all_connection_df = pd.read_csv("./data/all_connection_df.csv")
dfFilt = all_connection_df[['bodyId_pre', 'bodyId_post', 'weight']] # sknetwork uses 3rd col as weight

In [6]:
# sknetwork: only needed bc Louvain's clustering assignment output is based on graph.names order
graph = from_edge_list(list(dfFilt.itertuples(index=False)), weighted=True, directed=True) # without directed=True, wrong # of elements
graph.adjacency, graph.names

(<5596x5596 sparse matrix of type '<class 'numpy.int64'>'
 	with 389065 stored elements in Compressed Sparse Row format>,
 array([ 326253554,  357245785,  357249472, ..., 7112615304, 7112622763,
        7112624834]))

In [7]:
# create directed, weighted networkx graph from dataframe of edges & weights
# NOTE: it seems that NetworkX conversion shuffles edge ordering (so clustering results may be different)
G = nx.from_pandas_edgelist(all_connection_df, 'bodyId_pre', 'bodyId_post', 'weight', create_using=nx.DiGraph())
G.number_of_nodes(), G.number_of_edges()

(5596, 389065)

In [9]:
consensus500 = consensusResults[-1][1] # consensus clustering results for 500 iterations

In [15]:
# extract subgraphs for clusters 1,2,5
clusterIds = [1,2,5]
clusterSubgraphs = []

for clusterId in clusterIds:
    cluster_indices = np.where(consensus500 == clusterId)[0]
    cluster_nodes = [graph.names[i] for i in cluster_indices] # bodyIds
    cluster_subgraph = G.subgraph(cluster_nodes)
    print(cluster_subgraph.number_of_nodes(), cluster_subgraph.number_of_edges())
    # cluster_subgraph.name = f"cluster{clusterId}_iter500"
    clusterSubgraphs.append(cluster_subgraph)

1708 89695
1284 91524
1080 43135


In [14]:
# Iterating over subgraphs
for i, subG in enumerate(clusterSubgraphs):
    # save for Gephi
    nx.write_gexf(subG, Path.cwd() / "figures" / "gephi" / f"subgraph_cluster{clusterIds[i]}.gexf")